# 14_1.Logistic Regression(분류)
- https://www.statsmodels.org/stable/discretemod.html

## 1.기본 package 설정

In [ ]:
# 그래프에서 한글 폰트 인식하기
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
!pip install pingouin

#  *** 런타임 다시 시작

In [ ]:
# 1.기본
import numpy as np  # numpy 패키지 가져오기
import matplotlib.pyplot as plt # 시각화 패키지 가져오기
import seaborn as sns # 시각화

# 2.데이터 가져오기
import pandas as pd # csv -> dataframe으로 전환

# 3.통계분석 package
import pingouin as pg
from scipy import stats
import statsmodels.api as sm

In [ ]:
# 기본세팅
# 테마 설정
sns.set_theme(style = "darkgrid")

# 한글 인식
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False # -인식

## 2.데이터 불러오기

### 2.1 데이터 프레임으로 저장
- 원본데이터(csv)를 dataframe 형태로 가져오기(pandas)

In [ ]:
lr_df = pd.read_csv('https://raw.githubusercontent.com/leecho-bigdata/statistics-python/main/14_1.LR.csv', encoding="cp949")
lr_df.head()

,id,진단,반지름,질감,주변부,크기,평활도,조밀성,오목,대칭,프랙탈
0,1,1,18.0,10.4,122.8,1001.0,0.118,0.278,0.300,0.242,0.079
1,2,1,20.6,17.8,132.9,1326.0,0.085,0.079,0.087,0.181,0.057
2,3,1,19.7,21.3,130.0,1203.0,0.110,0.160,0.197,0.207,0.060
3,4,0,13.5,14.4,87.5,566.3,0.098,0.081,0.067,0.189,0.058
4,5,0,13.1,15.7,85.6,520.0,0.108,0.127,0.046,0.197,0.068


### 2.2 범주형 변수 처리
- 가변수 처리시 문자로 처리를 해야 변수명 구분이 쉬움

In [ ]:
lr_df['진단'].replace({0:'양성', 1:'악성'}, inplace=True)
lr_df['진단'] = lr_df['진단'].astype('category')
lr_df

,id,진단,반지름,질감,주변부,크기,평활도,조밀성,오목,대칭,프랙탈
0,1,악성,18.0,10.4,122.8,1001.0,0.118,0.278,0.300,0.242,0.079
1,2,악성,20.6,17.8,132.9,1326.0,0.085,0.079,0.087,0.181,0.057
2,3,악성,19.7,21.3,130.0,1203.0,0.110,0.160,0.197,0.207,0.060
3,4,양성,13.5,14.4,87.5,566.3,0.098,0.081,0.067,0.189,0.058
4,5,양성,13.1,15.7,85.6,520.0,0.108,0.127,0.046,0.197,0.068
...,...,...,...,...,...,...,...,...,...,...,...
560,561,악성,21.6,22.4,142.0,1479.0,0.111,0.116,0.244,0.173,0.056
561,562,악성,20.1,28.3,131.2,1261.0,0.098,0.103,0.144,0.175,0.055
562,563,악성,16.6,28.1,108.3,858.1,0.085,0.102,0.093,0.159,0.056
563,564,악성,20.6,29.3,140.1,1265.0,0.118,0.277,0.351,0.240,0.070


### 2.3 자료구조 살펴보기

In [ ]:
lr_df.shape

(565, 11)

In [ ]:
lr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565 entries, 0 to 564
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   id      565 non-null    int64   
 1   진단      565 non-null    category
 2   반지름     565 non-null    float64 
 3   질감      565 non-null    float64 
 4   주변부     565 non-null    float64 
 5   크기      565 non-null    float64 
 6   평활도     565 non-null    float64 
 7   조밀성     565 non-null    float64 
 8   오목      565 non-null    float64 
 9   대칭      565 non-null    float64 
 10  프랙탈     565 non-null    float64 
dtypes: category(1), float64(9), int64(1)
memory usage: 44.9 KB


In [ ]:
lr_df.columns

Index(['id', '진단', '반지름', '질감', '주변부', '크기', '평활도', '조밀성', '오목', '대칭', '프랙탈'], dtype='object')

## 3.기술통계

In [ ]:
# 그룹별 기술통계
lr_df.describe().round(3).T

,count,mean,std,min,25%,50%,75%,max
id,565.0,283.000,163.246,1.000,142.000,283.000,424.000,565.000
반지름,565.0,14.143,3.534,7.000,11.700,13.400,15.900,28.100
질감,565.0,19.292,4.310,9.700,16.200,18.800,21.800,39.300
주변부,565.0,92.034,24.368,43.800,75.200,86.300,104.300,188.500
크기,565.0,655.836,352.944,143.500,419.800,551.100,788.500,2501.000
평활도,565.0,0.096,0.014,0.053,0.086,0.096,0.105,0.163
조밀성,565.0,0.105,0.053,0.019,0.065,0.094,0.131,0.345
오목,565.0,0.089,0.080,0.000,0.030,0.062,0.132,0.427
대칭,565.0,0.181,0.027,0.106,0.162,0.179,0.196,0.304
프랙탈,565.0,0.063,0.007,0.050,0.058,0.062,0.066,0.097


In [ ]:
# 범주형 변수
# lecture_df.columns
categorical_features = ['진단']

for col in categorical_features:
    print("----", col, "----")
    results = lr_df[col].value_counts()
    print(results, "\n")

---- 진단 ----
양성    357
악성    208
Name: 진단, dtype: int64 



## 4.Logistic Regression
- https://www.statsmodels.org/stable/examples/notebooks/generated/ols.html
- 수치형 + 범주형
- dmatrix 사용


### 4.1 회귀분석

In [ ]:
# columns = ['반지름', '질감', '주변부', '크기', '평활도', '조밀성', '오목', '대칭', '프랙탈']

# 다중공선 제거: 반지름, 주변부
columns = ['질감', '크기', '평활도', '조밀성', '오목', '대칭', '프랙탈']

formula = "진단 ~ " + " + ".join(columns)
formula

'진단 ~ 질감 + 크기 + 평활도 + 조밀성 + 오목 + 대칭 + 프랙탈'

In [ ]:
# dmatrix 이용
from patsy import dmatrices

y, X = dmatrices(formula,
                 data = lr_df,
                 return_type = 'dataframe')

In [ ]:
X.head()

,Intercept,질감,크기,평활도,조밀성,오목,대칭,프랙탈
0,1.0,10.4,1001.0,0.118,0.278,0.300,0.242,0.079
1,1.0,17.8,1326.0,0.085,0.079,0.087,0.181,0.057
2,1.0,21.3,1203.0,0.110,0.160,0.197,0.207,0.060
3,1.0,14.4,566.3,0.098,0.081,0.067,0.189,0.058
4,1.0,15.7,520.0,0.108,0.127,0.046,0.197,0.068


In [ ]:
y = y.drop('진단[양성]', axis = 1)
y.head()

,진단[악성]
0,1.0
1,1.0
2,1.0
3,0.0
4,0.0


In [ ]:
model = sm.Logit(y, X)   # 모델 생성
result = model.fit()   # 모델 실행

Optimization terminated successfully.
         Current function value: 0.105936
         Iterations 11


In [ ]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:                 진단[악성]   No. Observations:                  565
Model:                          Logit   Df Residuals:                      557
Method:                           MLE   Df Model:                            7
Date:                Sat, 02 Mar 2024   Pseudo R-squ.:                  0.8390
Time:                        12:45:47   Log-Likelihood:                -59.854
converged:                       True   LL-Null:                       -371.75
Covariance Type:            nonrobust   LLR p-value:                1.835e-130
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -34.9140      7.612     -4.587      0.000     -49.833     -19.995
질감             0.4755      0.080      5.926      0.000       0.318       0.633
크기             0.0170      0.003      5.330      0.0

### 4.2 odds

In [ ]:
print("===== 계수 =====")
print(result.params)
print("\n")
print("===== odds =====")
print(np.exp(result.params))

===== 계수 =====
Intercept    -34.913962
질감             0.475503
크기             0.016976
평활도          152.491779
조밀성            0.445319
오목            29.012504
대칭            23.300555
프랙탈         -119.861261
dtype: float64


===== odds =====
Intercept    6.871619e-16
질감           1.608823e+00
크기           1.017121e+00
평활도          1.683984e+66
조밀성          1.560989e+00
오목           3.980800e+12
대칭           1.316142e+10
프랙탈          8.808781e-53
dtype: float64


## 5.가정검정
- https://ethanweed.github.io/pythonbook/05.04-regression.html#regressionnormality
- 잔차의 등분산성: Breusch-Pagan
- 잔차의 정규성: Jarque-Bera, Omnibus(D'Angostino's test)
- 독립성(자기상관): Durbin-Watson
- 다중공선성(VIF): Cond. No

### 5.1 다중 공선성
- VIF 10이상 삭제

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.values.shape[1])]
vif["features"] = X.columns
print(vif.round(1))

   VIF Factor   features
0       392.3  Intercept
1         1.2         질감
2         4.7         크기
3         2.2        평활도
4         8.6        조밀성
5         7.4         오목
6         1.8         대칭
7         4.3        프랙탈
